In [65]:
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt

El votante representativo se preocupa por el valor esperado de su función de utilidad  $\mathbb{E}_{t}^{P}(\Gamma_{t})$

<ul>
    <li>c representa el consumo del bien privado por parte de un ciudadano representativo.</li>
    <li>g es el consumo del bien público (per cápita)</li>
    <li>k es la inversión pública (per cápita)</li>
</ul>

In [47]:
a, b = 1, 1
beta = 0.95
# Parámetros temporales
t = 0  # Punto fijo en el tiempo
T = 100  # Final del período

In [48]:
# Valores aleatorios
alpha1 = np.random.binomial(1, 0.5, T-t)
alpha2 = np.random.binomial(1, 0.5, T-t)

q1 = np.random.binomial(20, 0.5, T-t)
q2 = np.random.binomial(20, 0.5, T-t)

epsilon = alpha1 + alpha2 # competencia del agente independiente entre agentes y en el tiempo
eta = q1 + q2 # Shock no relacionado con la competencia pero si con el liderazgo independiente entre agentes y en el tiempo

In [49]:
y = 100  # Valor exógeno

# Valores de elección, por el momento son aleatorios
tau = np.random.uniform(0, y, T-t) # tau <= y
k = np.array([np.random.uniform(0, tau_val) for tau_val in tau]) # tau - k >= 0

# Definiciones de c y g
c = y - tau # c >= 0
g = tau - k + epsilon # g >= 0

In [56]:
# Funciones
def U(c, g):
    return a*np.log(c) + b*np.log(g)

def V(k):
    return -np.log(k)

def Gamma(c, g, k, eta, beta, t, T):
    return [beta**(s-t) * (U(c[s-t], g[s-t]) + V(k[s-t]) + eta[s-t]) for s in range(t, T)]

def E_Gamma(X):
    return np.mean(X)


In [57]:
# Ejemplo de uso de la función Gamma de utilidad esperada del agente para T periodos
utilidad = Gamma(c, g, k, eta, beta, t, T)
print("Utilidades:", utilidad)
utilida_esperada = E_Gamma(utilidad)
print("Resultado de la utilidad esperada", utilida_esperada)



Utilidades: [19.267501483215717, 30.373422099806344, 23.081170262490296, 21.786681381788995, 17.882098771117068, 13.23761832713574, 16.243677790081723, 16.294309048847825, 19.610319596402334, 19.055342241051715, 15.405857973116913, 12.334438713106922, 11.703791091549766, 9.43666619195181, 11.359942702980192, 10.58770194415677, 8.302435961290707, 8.11312182254378, 11.473193757024962, 8.954222520552767, 7.937614901155448, 8.13322887935155, 7.222113612698594, 6.3028894281602, 6.8711143051517665, 6.479254834440031, 6.538600794572057, 5.066951752660338, 6.361245296099959, 5.466788834192165, 5.719731468442905, 5.544905197673133, 4.276848218112698, 4.353176217658516, 5.051320174150474, 3.782999647859132, 4.22385274613557, 4.0689480269154, 2.547003688844663, 2.915232284840432, 2.516980925643319, 2.00616921603838, 2.2687670682961616, 2.661459736064743, 3.3481139601862653, 2.149608887698399, 1.470560086645828, 2.160410578055159, 2.2310895633915258, 2.405631953149103, 1.8261574553436923, 2.215388

Función de utilidad del líder $\mathbf{E}_{t}^{I}(\Gamma_{t})+\sum_{s=t}^{T}\beta^{s-t}X\pi_{s,t}$

Función de elección del líder

\begin{equation}
    v_{t} =
    \begin{cases}
        1 &  \text{si } \mathbf{E}_{t}^{P}(\Gamma_{t+1}) \geq \mathbf{E}_{t}^{P}(\Gamma_{t+1}^{O})\\
        0 & \text{en otro caso}\\
\end{cases}
\end{equation}

## Equilibrio bajo información completa

\begin{equation}
    \begin{aligned}
        \max_{\tau, g} & U(y-\tau, g)+\beta V(\tau + \epsilon - g) \\
        \text{sujeto a } & g, y- \tau, \tau + \epsilon-g \geq 0 \\
   \end{aligned}
\end{equation}

### En $t=\bar{t}$

In [66]:
# Parámetros
a, b = 1, 1
beta = 0.95

y = 100  # Valor exógeno

In [67]:
# Valores aleatorios
alpha1 = np.random.binomial(1, 0.5, T-t)
alpha2 = np.random.binomial(1, 0.5, T-t)

q1 = np.random.binomial(20, 0.5, T-t)
q2 = np.random.binomial(20, 0.5, T-t)

epsilon = alpha1 + alpha2 # competencia del agente independiente entre agentes y en el tiempo
eta = q1 + q2 # Shock no relacionado con la competencia pero si con el liderazgo independiente entre agentes y en el tiempo

In [68]:
# Funciones estáticas de utilidad
def U(c, g):
    return a*np.log(c) + b*np.log(g)

def V(k):
    return -np.log(k)

In [73]:
import numpy as np
from scipy.optimize import minimize

# Objective function to maximize
def objective_function(x, y, epsilon, beta, a, b):
    tau, g = x
    c = y - tau
    k = tau + epsilon - g
    return -(a * np.log(c) + b * np.log(g) + beta * (-np.log(k)))  # Negative because we are using a minimizer

# Constraints
def constraint1(x, y):
    # y - tau >= 0
    return y - x[0]

def constraint2(x):
    # g >= 0
    return x[1]

def constraint3(x, epsilon):
    # tau + epsilon - g >= 0
    return x[0] + epsilon - x[1]

# Set parameters
a, b = 1, 1
beta = 0.95
y = 100

# Example values for epsilon
epsilon_example = np.random.choice([0, 10, 20])

# Constraints
con1 = {'type': 'ineq', 'fun': constraint1, 'args': (y,)}
con2 = {'type': 'ineq', 'fun': constraint2}
con3 = {'type': 'ineq', 'fun': constraint3, 'args': (epsilon_example,)}

# Combine constraints
constraints = [con1, con2, con3]

# Initial guess (start from the middle of the allowed range for tau and g)
x0 = [y/2, y/4]

# Perform optimization
result = minimize(objective_function, x0, args=(y, epsilon_example, beta, a, b), constraints=constraints)

result.x, -result.fun  # Optimal tau and g, and the maximum value of the objective function


C:\Users\OscarJaramillo\AppData\Local\Temp\ipykernel_30460\3146507759.py:9: RuntimeWarning: invalid value encountered in log
  return -(a * np.log(c) + b * np.log(g) + beta * (-np.log(k)))  # Negative because we are using a minimizer


(array([31.84811569, 51.84811569]), 38.075397337432136)

In [74]:
# Setting the time period t
T = 10

# Function to perform the optimization for each time period
def optimize_for_each_time_period(T, y, beta, a, b):
    optimal_tau = []
    optimal_g = []
    max_values = []

    for t in range(T):
        # Generate random values for epsilon
        epsilon_t = np.random.choice([0, 10, 20])

        # Constraints for this time period
        con1 = {'type': 'ineq', 'fun': constraint1, 'args': (y,)}
        con2 = {'type': 'ineq', 'fun': constraint2}
        con3 = {'type': 'ineq', 'fun': constraint3, 'args': (epsilon_t,)}

        # Combine constraints
        constraints_t = [con1, con2, con3]

        # Perform optimization for this time period
        result_t = minimize(objective_function, x0, args=(y, epsilon_t, beta, a, b), constraints=constraints_t)

        # Store the results
        optimal_tau.append(result_t.x[0])
        optimal_g.append(result_t.x[1])
        max_values.append(-result_t.fun)

    return optimal_tau, optimal_g, max_values

# Perform the optimization for T time periods
optimal_tau_list, optimal_g_list, max_values_list = optimize_for_each_time_period(T, y, beta, a, b)

optimal_tau_list, optimal_g_list, max_values_list


C:\Users\OscarJaramillo\AppData\Local\Temp\ipykernel_30460\3146507759.py:9: RuntimeWarning: invalid value encountered in log
  return -(a * np.log(c) + b * np.log(g) + beta * (-np.log(k)))  # Negative because we are using a minimizer


([31.848115690491518,
  35.519987191204926,
  31.848115690491518,
  39.30665359442016,
  31.848115690491518,
  39.30665359442016,
  35.519987191204926,
  39.30665359442016,
  31.848115690491518,
  35.519987191204926],
 [51.8481156904915,
  45.51998719120103,
  51.8481156904915,
  39.30665359441338,
  51.8481156904915,
  39.30665359441338,
  45.51998719120103,
  39.30665359441338,
  51.8481156904915,
  45.51998719120103],
 [38.075397337432136,
  32.94256689467652,
  38.075397337432136,
  32.208619008979404,
  38.075397337432136,
  32.208619008979404,
  32.94256689467652,
  32.208619008979404,
  38.075397337432136,
  32.94256689467652])